In [1]:
import mlrun
project = mlrun.get_or_create_project(name='mlops', user_project=True, context='./')

> 2023-01-14 18:53:50,685 [info] loaded project mlops from MLRun DB


### Setting functions

In [2]:
import os

# Setting get data function
get_data = mlrun.code_to_function(name='gen_dataset', kind='job', image='mlrun/mlrun', handler='get_data', filename='src/get_data.py')

# Mount it:
get_data.apply(mlrun.auto_mount())
if os.getenv('V3IO_ACCESS_KEY','False')=='False':
    get_data.spec.disable_auto_mount=False
    
project.set_function(get_data)

# Setting dalex function
dalex = mlrun.code_to_function(name='dalex', kind='job', handler='run_dalex',filename='src/dalex.py')

project.set_function(dalex)

# Setting training function
dalex = mlrun.code_to_function(name='train', kind='job', handler='train',filename='src/auto_trainer.py')
project.set_function(dalex)

In [6]:
%%writefile src/trainer_baseline.py
import mlrun
from kfp import dsl
import sklearn
import json

@dsl.pipeline(
    name="Automatic Pipeline",
    description="Train & Evaluate"
)
def kfpipeline(dataset: str='housing',
               path: str='/home/jovyan/data/src/housing.csv',
               label_column:str='MEDV'):
    
    project = mlrun.get_current_project()
        
    get_data_run = mlrun.run_function(name='get_data',
                                      function='gen-dataset',
                                      params={'dataset': dataset,
                                              'path': path},
                                      outputs=[dataset])

    
    dalex = project.run_function(name='dalex',
                                 function='dalex',
                                 params={'df_train': get_data_run.outputs[dataset],
                                         'target': label_column},
                                 outputs=['train_data', 'test_data', 'dalex_output'])
    
    
    params = {"model_class": "xgboost.XGBRegressor",
              "label_columns": label_column,
              "model_name": dataset + '_dalex'}

    for key,val in json.loads(mlrun.get_dataitem(dalex.outputs['dalex_output']).get()).items():
        params['sample_weight'] = val
    
    # Train a model using the auto_trainer hub function
    train_run = mlrun.run_function(name= 'train',
                                   function='train',
                                   inputs={"dataset": dalex.outputs['train_data'], 
                                           "test_set": dalex.outputs['test_data']},
                                   params = params, 
                                   handler='train',
                                   outputs=["model"],
                               )

Overwriting src/trainer_baseline.py


In [7]:
# Register the workflow file:
workflow_name = "trainer_baseline"
project.set_workflow(workflow_name, "src/trainer_baseline.py")

# Save the project:
project.save()

In [ ]:
run = project.run(name=workflow_name,watch=False,local=True, overwrite=True,
                  arguments={'path': '/home/jovyan/data/MLOps22/project/src/housing.csv'})

In [ ]:
run = project.run(name=workflow_name,watch=False,local=True, overwrite=True,
                  arguments={'dataset': 'motor',
                             'path': '/home/jovyan/data/MLOps22/project/src/freMTPL2freq.csv',
                             'label_column': 'ClaimNb'})

In [ ]:
project.list_artifacts()